In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [3]:
data=  [['2018-08-01|2017-06-01|2015-11-01|2012-08-21|2010-04-02'],
       ['2016-08-11|2013-08-25|2013-04-01|2012-01-01']]
columns= ['date']
df= spark.createDataFrame(data,columns)
display(df)

date
2018-08-01|2017-06-01|2015-11-01|2012-08-21|2010-04-02
2016-08-11|2013-08-25|2013-04-01|2012-01-01


In [4]:
from pyspark.sql import functions as F
df.withColumn("date", F.split(F.regexp_replace("date","\|",","),',')).withColumn("start", F.reverse(F.col("date"))[0]).withColumn("end",F.col("date")[0]).withColumn("total_time", F.datediff("end","start")).withColumn("Average", F.col("total_time")/(F.size(F.col("date")))).drop("start","end").show()


+--------------------+----------+-------+
 date|total_time|Average|
+--------------------+----------+-------+
[2018-08-01, 2017...| 3043| 608.6|
[2016-08-11, 2013...| 1684| 421.0|
+--------------------+----------+-------+

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [6]:
data=  [[100,1],
        [101,1],  
        [102,1],
        [110,2],
        [104,2],
        [105,12],
        [106,12],
        [107,1],
        [108,2],
        [109,12],
        [103,2],
        [111,0],
        [112,22],
        [113,17],
       [114,20]]
columns= ['SecondsInHour','Total']
df= spark.createDataFrame(data,columns)
df.show()

+-------------+-----+
SecondsInHour|Total|
+-------------+-----+
 100| 1|
 101| 1|
 102| 1|
 110| 2|
 104| 2|
 105| 12|
 106| 12|
 107| 1|
 108| 2|
 109| 12|
 103| 2|
 111| 0|
 112| 22|
 113| 17|
 114| 20|
+-------------+-----+

In [7]:
w2= Window().orderBy(df.SecondsInHour)
df.withColumn("lagdiff",(F.col("Total")- F.lag("Total").over(w2)))\
.withColumn("lagdiff2", F.col("Total")- F.lag("Total").over(w2))\
.withColumn("lagdiff3", F.when((F.col("lagdiff2")>0),F.lit(0)).otherwise(F.col("lagdiff2")*-1))\
.withColumn("lagdiff4", F.when((F.col("lagdiff") <0), F.lit(0)).otherwise(F.col("lagdiff")))\
.withColumn("In", F.when(F.col("lagdiff").isNull(), F.col("Total")).otherwise(F.col("lagdiff4")))\
.withColumn("Out",F.when(F.col("lagdiff3").isNull(), F.lit(0)).otherwise(F.col("lagdiff3")))\
.orderBy(df.SecondsInHour.asc())\
.drop("lagdiff","lagdiff2","lagdiff3","lagdiff4").show()

+-------------+-----+---+---+
SecondsInHour|Total| In|Out|
+-------------+-----+---+---+
 100| 1| 1| 0|
 101| 1| 0| 0|
 102| 1| 0| 0|
 103| 2| 1| 0|
 104| 2| 0| 0|
 105| 12| 10| 0|
 106| 12| 0| 0|
 107| 1| 0| 11|
 108| 2| 1| 0|
 109| 12| 10| 0|
 110| 2| 0| 10|
 111| 0| 0| 2|
 112| 22| 22| 0|
 113| 17| 0| 5|
 114| 20| 3| 0|
+-------------+-----+---+---+